In [1]:
import json
import numpy as np
import pandas as pd
import keras
import os
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import text_to_word_sequence
from keras.models import Sequential, Model, load_model
from keras.layers import Dense, Activation, Dropout, Input, Flatten, Embedding, LSTM
from keras.utils import np_utils
from keras.utils import to_categorical
from sklearn import preprocessing
import sklearn
import time
from random import shuffle
import pickle

c:\users\kevin\anaconda3\envs\tensorflow\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
os.getcwd()

'E:\\GT\\JrDesign\\demo\\grocery_e-receipt\\LSTM + Embedding'

In [3]:
with open("../synthetic_data/final_train_labels.json") as f:
    data = json.load(f)


In [4]:
x = [ea[1] for ea in data]
y = [ea[0] for ea in data]
df = pd.DataFrame(data={"X": x, "Y": y})
print (len(df))
df.head()

112556


,X,Y
0,FLD PRM BCN,FLAVORED PREMIUM BACON
1,10% NTL FLX SDS,100% NATURAL FLAX SEEDS
2,RGC BLK TRS DRK CHT,ORGANIC BLACK TRUFFLES DARK CHOCOLATE
3,WLD DRK WTH DDD VTN C PP' PNH RTY FLD,WILD DRINK WITH ADDED VITAMIN C POPPIN' PUNCH ...
4,SN STL NDL,ASIAN STYLE NOODLE


In [5]:
# Y Vocab
labels = y
le = preprocessing.LabelEncoder()
df['encoded_y'] = le.fit_transform(y)
print ("# categories = ", max(df.encoded_y.tolist())+1)
df.head(5)

# categories =  28139


,X,Y,encoded_y
0,FLD PRM BCN,FLAVORED PREMIUM BACON,8681
1,10% NTL FLX SDS,100% NATURAL FLAX SEEDS,99
2,RGC BLK TRS DRK CHT,ORGANIC BLACK TRUFFLES DARK CHOCOLATE,16083
3,WLD DRK WTH DDD VTN C PP' PNH RTY FLD,WILD DRINK WITH ADDED VITAMIN C POPPIN' PUNCH ...,27775
4,SN STL NDL,ASIAN STYLE NOODLE,1396


In [6]:
le.inverse_transform([8681])

array(['FLAVORED PREMIUM BACON'], dtype='<U132')

In [7]:
# X Vocab
MAX_SEQ_LEN = 25
MAX_NUM_WORDS = 20000
EMBEDDING_DIM = 256

flattened = np.asarray(x).reshape(-1)
t = Tokenizer(num_words=MAX_NUM_WORDS)
t.fit_on_texts(flattened)

## see encoded X --> do batch encoding, cannot load into memory
encoded_x = t.texts_to_sequences(flattened)
train_x = pad_sequences(encoded_x, maxlen=MAX_SEQ_LEN)
train_x

array([[   0,    0,    0, ...,  294,   45,  270],
       [   0,    0,    0, ...,  180,  551,  308],
       [   0,    0,    0, ...,  476,  101,   64],
       ...,
       [   0,    0,    0, ...,  361,   85,   18],
       [   0,    0,    0, ...,    0, 1521,  240],
       [   0,    0,    0, ...,    0,  617,    8]])

In [8]:
train_y = to_categorical(df.encoded_y.tolist(), num_classes=max(df.encoded_y.tolist())+1)
train_y

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [9]:
model = Sequential()
model.add(Embedding(MAX_NUM_WORDS, EMBEDDING_DIM, input_length=MAX_SEQ_LEN))
model.add(LSTM(100))
model.add(Dense(len(train_y[0]), activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 25, 256)           5120000   
_________________________________________________________________
lstm_1 (LSTM)                (None, 100)               142800    
_________________________________________________________________
dense_1 (Dense)              (None, 28139)             2842039   
Total params: 8,104,839
Trainable params: 8,104,839
Non-trainable params: 0
_________________________________________________________________
None


In [10]:
print (train_x.shape)
print (train_y.shape)

(112556, 25)
(112556, 28139)


In [11]:
start = time.time()
model.fit(train_x, train_y, epochs=6)
model.save("lstm_6_epochs.h5")
print ("Took: ", time.time()-start)

Epoch 1/6
112556/112556 [==============================] - 325s 3ms/step - loss: 10.1183 - acc: 2.3100e-04
Epoch 2/6
112556/112556 [==============================] - 300s 3ms/step - loss: 7.7755 - acc: 0.0435
Epoch 3/6
112556/112556 [==============================] - 327s 3ms/step - loss: 5.2500 - acc: 0.2490
Epoch 4/6
112556/112556 [==============================] - 335s 3ms/step - loss: 3.3694 - acc: 0.5075
Epoch 5/6
112556/112556 [==============================] - 319s 3ms/step - loss: 2.1268 - acc: 0.6799
Epoch 6/6
112556/112556 [==============================] - 330s 3ms/step - loss: 1.4101 - acc: 0.7739
Took:  1938.5239877700806


# Evaluation

In [12]:
test_num = 300
preds = model.predict(train_x[:test_num])
pred_labels = [[np.argmax(x)] for x in preds]

In [13]:
pred_titles = le.inverse_transform(pred_labels)
pred_titles

c:\users\kevin\anaconda3\envs\tensorflow\lib\site-packages\sklearn\preprocessing\label.py:273: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


array(['FLAVORED PREMIUM BACON', '100% NATURAL FLAX SEEDS',
       'ORGANIC BLACK TRUFFLES DARK CHOCOLATE',
       "WILD DRINK WITH ADDED VITAMIN C POPPIN' PUNCH ARTIFICIALLY FLAVORED",
       'ASIAN STYLE NOODLE', 'PEI TIEN ENERGY 99 STICKS CHOCOLATE',
       'CO-JACK', 'BACON BITES', 'CRINKLE CUT PICKLED BEETS',
       'ORGANIC AGAVE AMBER', 'SPLENDA', 'MILK CHOCOLATE & SURPRISE',
       'CHICKEN BREAST TENDERLOINS',
       'DOLMAS STUFFED GRAPE LEAVES WITH RAISINS & PINE NUTS',
       'CAJUN CHICKEN', 'DECAFFEINATED SAVORY', 'ITALIAN WAFFLE COOKIES',
       'MILD INDIAN CURRY SAUCE', 'ROAST BEEF SUB', 'SLICED BACON',
       'RICE & BLACK BEANS', 'DOWN HOME BBQ',
       'PETITE DICED TOMATOES WITH GARLIC & OLIVE OIL',
       'LIGHT CAESAR DRESSING', 'ORGANIC FARRO',
       'FIRE ROASTED SWEET RED PEPPERS', 'SEITAN ITALIAN HERB',
       'MASALA SIMMER SAUCE', 'CINNAMON DISCS', 'WHITE RICE LOAF',
       'CREAM OF CHICKEN CONDENSED SOUP', 'ORGANIC EXTRA FIRM TOFUPLUS',
       'LAL MASSO

In [14]:
analysis_df = pd.DataFrame({'pred_x': pred_titles, 'x': x[:test_num], 'y': y[:test_num]})
analysis_df = analysis_df[['x', 'y', 'pred_x']]

In [15]:
analysis_df

,x,y,pred_x
0,FLD PRM BCN,FLAVORED PREMIUM BACON,FLAVORED PREMIUM BACON
1,10% NTL FLX SDS,100% NATURAL FLAX SEEDS,100% NATURAL FLAX SEEDS
2,RGC BLK TRS DRK CHT,ORGANIC BLACK TRUFFLES DARK CHOCOLATE,ORGANIC BLACK TRUFFLES DARK CHOCOLATE
3,WLD DRK WTH DDD VTN C PP' PNH RTY FLD,WILD DRINK WITH ADDED VITAMIN C POPPIN' PUNCH ...,WILD DRINK WITH ADDED VITAMIN C POPPIN' PUNCH ...
4,SN STL NDL,ASIAN STYLE NOODLE,ASIAN STYLE NOODLE
5,P TN NRY 99 STS CHT,PEI TIEN ENERGY 99 STICKS CHOCOLATE,PEI TIEN ENERGY 99 STICKS CHOCOLATE
6,C-K,CO-JACK,CO-JACK
7,BCN BTS,BACON BITES,BACON BITES
8,CRL CT PCD BTS,CRINKLE CUT PICKLED BEETS,CRINKLE CUT PICKLED BEETS
9,RGC GV MBR,ORGANIC AGAVE AMBER,ORGANIC AGAVE AMBER
